<a href="https://www.kaggle.com/code/yeshavyas/speech-emotion-recognition-ser?scriptVersionId=173187956" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import os
import librosa
import re
# 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
# 
from sklearn import preprocessing, svm
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# 
import tensorflow as tf


In [2]:
Ravdess = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
Crema = "/kaggle/input/cremad/AudioWAV/"
Savee = "/kaggle/input/surrey-audiovisual-expressed-emotion-savee/ALL/"
Tess ="/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"

In [3]:
ravdess_directory_list = os.listdir(Ravdess)

emotion_df = []

for dir in ravdess_directory_list:
    actor = os.listdir(os.path.join(Ravdess, dir))
    for wav in actor:
        info = wav.partition(".wav")[0].split("-")
        emotion = int(info[2])
        emotion_df.append((emotion, os.path.join(Ravdess, dir, wav)))

Ravdess_df = pd.DataFrame.from_dict(emotion_df)
Ravdess_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

Ravdess_df.Emotion.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotion,Path
0,surprise,/kaggle/input/ravdess-emotional-speech-audio/a...
1,neutral,/kaggle/input/ravdess-emotional-speech-audio/a...
2,disgust,/kaggle/input/ravdess-emotional-speech-audio/a...
3,disgust,/kaggle/input/ravdess-emotional-speech-audio/a...
4,neutral,/kaggle/input/ravdess-emotional-speech-audio/a...


In [4]:
emotion_df = []

for wav in os.listdir(Crema):
    info = wav.partition(".wav")[0].split("_")
    if info[2] == 'SAD':
        emotion_df.append(("sad", Crema + "/" + wav))
    elif info[2] == 'ANG':
        emotion_df.append(("angry", Crema + "/" + wav))
    elif info[2] == 'DIS':
        emotion_df.append(("disgust", Crema + "/" + wav))
    elif info[2] == 'FEA':
        emotion_df.append(("fear", Crema + "/" + wav))
    elif info[2] == 'HAP':
        emotion_df.append(("happy", Crema + "/" + wav))
    elif info[2] == 'NEU':
        emotion_df.append(("neutral", Crema + "/" + wav))
    else:
        emotion_df.append(("unknown", Crema + "/" + wav))


Crema_df = pd.DataFrame.from_dict(emotion_df)
Crema_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

Crema_df.head()

,Emotion,Path
0,disgust,/kaggle/input/cremad/AudioWAV//1028_TSI_DIS_XX...
1,happy,/kaggle/input/cremad/AudioWAV//1075_IEO_HAP_LO...
2,happy,/kaggle/input/cremad/AudioWAV//1084_ITS_HAP_XX...
3,disgust,/kaggle/input/cremad/AudioWAV//1067_IWW_DIS_XX...
4,disgust,/kaggle/input/cremad/AudioWAV//1066_TIE_DIS_XX...


In [5]:
tess_directory_list = os.listdir(Tess)

emotion_df = []

for dir in tess_directory_list:
    for wav in os.listdir(os.path.join(Tess, dir)):
        info = wav.partition(".wav")[0].split("_")
        emo = info[2]
        if emo == "ps":
            emotion_df.append(("surprise", os.path.join(Tess, dir, wav)))
        else:
            emotion_df.append((emo, os.path.join(Tess, dir, wav)))


Tess_df = pd.DataFrame.from_dict(emotion_df)
Tess_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

Tess_df.head()

,Emotion,Path
0,fear,/kaggle/input/toronto-emotional-speech-set-tes...
1,fear,/kaggle/input/toronto-emotional-speech-set-tes...
2,fear,/kaggle/input/toronto-emotional-speech-set-tes...
3,fear,/kaggle/input/toronto-emotional-speech-set-tes...
4,fear,/kaggle/input/toronto-emotional-speech-set-tes...


In [6]:
savee_directiory_list = os.listdir(Savee)

emotion_df = []

for wav in savee_directiory_list:
    info = wav.partition(".wav")[0].split("_")[1].replace(r"[0-9]", "")
    emotion = re.split(r"[0-9]", info)[0]
    if emotion=='a':
        emotion_df.append(("angry", Savee + "/" + wav))
    elif emotion=='d':
        emotion_df.append(("disgust", Savee + "/" + wav))
    elif emotion=='f':
        emotion_df.append(("fear", Savee + "/" + wav))
    elif emotion=='h':
        emotion_df.append(("happy", Savee + "/" + wav))
    elif emotion=='n':
        emotion_df.append(("neutral", Savee + "/" + wav))
    elif emotion=='sa':
        emotion_df.append(("sad", Savee + "/" + wav))
    else:
        emotion_df.append(("surprise", Savee + "/" + wav))


Savee_df = pd.DataFrame.from_dict(emotion_df)
Savee_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

Savee_df.head()

,Emotion,Path
0,happy,/kaggle/input/surrey-audiovisual-expressed-emo...
1,fear,/kaggle/input/surrey-audiovisual-expressed-emo...
2,happy,/kaggle/input/surrey-audiovisual-expressed-emo...
3,disgust,/kaggle/input/surrey-audiovisual-expressed-emo...
4,angry,/kaggle/input/surrey-audiovisual-expressed-emo...


In [7]:
path_df = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis=0, ignore_index=True)
print(path_df.shape)
features_path = "./all_datasets_path_emotions.csv"
path_df.to_csv(features_path, index=False)

(12162, 2)


In [ ]:
n_fft = 2048
hop_length = 512

def chunks(data, frame_length, hop_length):
    for i in range(0, len(data), hop_length):
        yield data[i:i+frame_length]

# Zero Crossing Rate
def zcr(data, frame_length=2048, hop_length=512):
    zcr = librosa.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(zcr)


def energy(data, frame_length=2048, hop_length=512):
    en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])
    return en / frame_length


def rmse(data, frame_length=2048, hop_length=512):
    rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(rmse)


def entropy_of_energy(data, frame_length=2048, hop_length=512):
    energies = energy(data, frame_length, hop_length)
    energies /= np.sum(energies)

    entropy = 0.0
    entropy -= energies * np.log2(energies)
    return entropy


def spc(data, sr, frame_length=2048, hop_length=512):
    spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sr, n_fft=frame_length, hop_length=hop_length)
    return np.squeeze(spectral_centroid)

def spc_flux(data):
    isSpectrum = data.ndim == 1
    if isSpectrum:
        data = np.expand_dims(data, axis=1)

    X = np.c_[data[:, 0], data]
    af_Delta_X = np.diff(X, 1, axis=1)
    vsf = np.sqrt((np.power(af_Delta_X, 2).sum(axis=0))) / X.shape[0]

    return np.squeeze(vsf) if isSpectrum else vsf


def spc_rollof(data, sr, frame_length=2048, hop_length=512):
    spcrollof = librosa.feature.spectral_rolloff(y=data, sr=sr, n_fft=frame_length, hop_length=hop_length)
    return np.squeeze(spcrollof)


def chroma_stft(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    stft = np.abs(librosa.stft(data))
    chroma_stft = librosa.feature.chroma_stft(S=stft, sr=sr)
    return np.squeeze(chroma_stft.T) if not flatten else np.ravel(chroma_stft.T)

def chroma_cens(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    chroma_cens = librosa.feature.chroma_cens(y=x, sr=sr)
    return np.squeeze(chroma_cens.T) if not flatten else np.ravel(chroma_cens.T)
    
def mel_spc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    mel = librosa.feature.melspectrogram(y=data, sr=sr)
    return np.squeeze(mel.T) if not flatten else np.ravel(mel.T)

def mfcc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    mfcc_feature = librosa.feature.mfcc(y=data, sr=sr)
    return np.squeeze(mfcc_feature.T) if not flatten else np.ravel(mfcc_feature.T)

def spec_bandwidth(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    spec_btw  = librosa.feature.spectral_bandwidth(y=data, sr=sr)
    return np.squeeze(spec_btw)

def spec_contrast(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    stft = np.abs(librosa.stft(data))
    spec_cont = librosa.feature.spectral_contrast(S=stft,sr=sr)
    return np.squeeze(spec_cont.T) if not flatten else np.ravel(spec_cont.T)

def spec_flat(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    spec_f = librosa.feature.spectral_flatness(y=data)
    return np.squeeze(spec_f)


# for a single audio file
path = np.array(path_df["Path"])[658]
data, sampling_rate = librosa.load(path, duration=2.5, offset=0.6)
len(data)

print("ZCR: ", zcr(data).shape)
print("Energy: ", energy(data).shape)
print("Entropy of Energy :", entropy_of_energy(data).shape)
print("RMS :", rmse(data).shape)
print("Spectral Centroid :", spc(data, sampling_rate).shape)
# print("Spectral Entropy: ", spc_entropy(data, sampling_rate).shape)
print("Spectral Flux: ", spc_flux(data).shape)
print("Spectral Rollof: ", spc_rollof(data, sampling_rate).shape)
print("Chroma STFT: ", chroma_stft(data, sampling_rate).shape)
print("MelSpectrogram: ", mel_spc(data, sampling_rate).shape)
print("MFCC: ", mfcc(data, sampling_rate).shape)

In [1]:
# functions for augmentation
def noise(data, random=False, rate=0.035, threshold=0.075):
    """Add some noise to sound sample. Use random if you want to add random noise with some threshold.
    Or use rate Random=False and rate for always adding fixed noise."""
    if random:
        rate = np.random.random() * threshold
    noise_amp = rate*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    """Stretching data with some rate."""
    return librosa.effects.time_stretch(data, rate=rate)

def shift(data, rate=1000):
    """Shifting data with some rate"""
    shift_range = int(np.random.uniform(low=-5, high = 5)*rate)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7, random=False):
    """"Add some pitch to sound sample. Use random if you want to add random pitch with some threshold.
    Or use pitch_factor Random=False and rate for always adding fixed pitch."""
    if random:
        pitch_factor=np.random.random() * pitch_factor
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)

In [ ]:
def extract_features(data, sr, frame_length=2048, hop_length=512):
    result = np.array([])
    result = np.hstack((result,
                        zcr(data, frame_length, hop_length),
                        # np.mean(energy(data, frame_length, hop_length),axis=0),
                        # np.mean(entropy_of_energy(data, frame_length, hop_length), axis=0),
                        rmse(data, frame_length, hop_length),
                        # spc(data, sr, frame_length, hop_length),
                        # spc_entropy(data, sr),
                        # spc_flux(data),
                        # spc_rollof(data, sr, frame_length, hop_length),
                        # chroma_stft(data, sr, frame_length, hop_length),
                        # mel_spc(data, sr, frame_length, hop_length, flatten=True)
                        mfcc(data, sr, frame_length, hop_length)
                                    ))
    return result

def get_features(path, duration=2.5, offset=0.6):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=duration, offset=offset)

     # without augmentation
    res1 = extract_features(data, sample_rate)
    result = np.array(res1)

    # data with noise
    noise_data = noise(data, random=True)
    res2 = extract_features(noise_data, sample_rate)
    result = np.vstack((result, res2)) # stacking vertically

    # data with pitching
    pitched_data = pitch(data, sample_rate, random=True)
    res3 = extract_features(pitched_data, sample_rate)
    result = np.vstack((result, res3)) # stacking vertically

    # data with pitching and white_noise
    new_data = pitch(data, sample_rate, random=True)
    data_noise_pitch = noise(new_data, random=True)
    res3 = extract_features(data_noise_pitch, sample_rate)
    result = np.vstack((result, res3)) # stacking vertically

    return result

X, Y = [], []
print("Feature processing...")
for path, emotion, ind in zip(path_df.Path, path_df.Emotion, range(path_df.Path.shape[0])):
    features = get_features(path)
    if ind % 100 == 0:
        print(f"{ind} samples has been processed...")
    for ele in features:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)
print("Done.")
features_path = "./zcr_rmse_mfcc_plus_random_aug_features.csv"
extracted_df = pd.DataFrame(X)
extracted_df["labels"] = Y
extracted_df.to_csv(features_path, index=False)
extracted_df.head()
# extracted_df = pd.read_csv(features_path)
# print(extracted_df.shape)

In [ ]:
features_path = "./zcr_rmse_mfcc_plus_random_aug_features.csv"

df = pd.read_csv(features_path)
df

In [ ]:
# specific for ravdess
# ser_df.Emotions.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust',8:'surprise'},inplace=True)
 
y = np.array(df['labels'])
l_encode = LabelEncoder()
l_encode.fit(y)
y = l_encode.transform(y)
y.shape
df.drop(columns=['labels'], inplace=True)


In [ ]:
df = df.fillna(0)

scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(df.values)
X_scaled.shape


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=30)

X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=30)

In [ ]:
def get_model_performance(clf, X_train=X_train, y_train=y_train, X_cv=X_cv, y_cv=y_cv):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_cv)
    accuracy = accuracy_score(y_cv, y_pred)
    report = classification_report(y_cv, y_pred)
    other_performance_params = precision_recall_fscore_support(y_cv, y_pred, average="macro")

    return accuracy, report, other_performance_params


In [ ]:
# # # using logistic regression
# clf6 = LogisticRegression(multi_class='auto', solver='lbfgs', max_iter=1000)
# # using decision tree 
# clf4 = DecisionTreeClassifier()

# Using support vector machine classifier
clf = svm.SVC(C=7)

# using k-nearest-neighbours
# clf5 = KNeighborsClassifier()

# #  using bagging ensemble learning
# clf3 = BaggingClassifier(estimator=svm.SVC(C=7), n_estimators=20, random_state=0)

# using boosting ensemble
# clf1 = HistGradientBoostingClassifier(max_iter=100)
# # using voting 
# clf = VotingClassifier(estimators=[('histgr', clf1), ('svc', clf2),('bagclf', clf3), ('dt', clf4), ('knn',clf5)], voting='hard')


# # using stacking ensemble learning
# def get_stacking():
#     # define the base models
#     level0 = list()

#     level0.append(('knn', KNeighborsClassifier()))
#     level0.append(('svc', svm.SVC(C=7)))
# #     level0.append(('logreg', LogisticRegression(multi_class='auto', solver='lbfgs', max_iter=1000)))
#     level0.append(('bagclf', BaggingClassifier(estimator=svm.SVC(C=7),n_estimators=20, random_state=0)))
# #     level0.append(('vboostclf', VotingClassifier(estimators=[('hgbc', clf1), ('svc', clf2), ('bgclf', clf3)], voting='hard')))
#     # define meta learner model
#     level1 = svm.SVC(C=7)
#     # define the stacking ensemble
#     model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
#     return model

# clf = get_stacking()

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# universal to all sklearn models
test_accuracy, report, other_performance_params = get_model_performance(clf)
train_accuracy = clf.score(X_train, y_train)
print(f"{type(clf).__name__}\n Test Accuracy: {test_accuracy} \n Train Accuracy: {train_accuracy}\n Classification report \n{report} \n Precision Recall fscore support \n {other_performance_params}")


In [ ]:
# harmonic and percussive components 
from tqdm import tqdm
import librosa as lb
def feature_extractor_harmonic_percussive(path):
    data, simple_rate = lb.load(path)
    D = lb.stft(data)
    #melspec_harmonic, melspec_percussive = lb.decompose.hpss(D)
    #melspec_harmonic = np.mean(melspec_harmonic)
    #melspec_percussive = np.mean(melspec_percussive)
    #logspec_hp = np.average([melspec_harmonic, melspec_percussive])
    harmonic = lb.effects.harmonic(data,margin=5.0)
    percussive = librosa.effects.percussive(data, margin=5.0)
    #harmonic = np.mean(harmonic)
    #percussive = np.mean(percussive)
    logspec_hp = np.average([harmonic, percussive])
    return logspec_hp

def feature_extractor_logmelspectrogram(path):
    data, simple_rate = lb.load(path)
    data = lb.feature.melspectrogram(data)
    data = lb.power_to_db(data)
    data = np.mean(data,axis=1)
    return data



x_harmonic_percussive, x_logmelspectrogram, y = [], [], []
for path in tqdm(files):
    file_name = path.split('\\')[-1] #Return File Name
    class_label = path.split('-')[-1] #Return Class Id
    class_label = class_label.split('.')[0]
        
    x_harmonic_percussive.append(feature_extractor_harmonic_percussive(path))
    x_logmelspectrogram.append(feature_extractor_logmelspectrogram(path))
    y.append(class_label)

x_harmonic_percussive = np.array(x_harmonic_percussive)
x_logmelspectrogram = np.array(x_logmelspectrogram)
y = np.array(y)
features = np.hstack((x_harmonic_percussive.reshape(len(x_harmonic_percussive), 1), x_logmelspectrogram))
labels = y
features.shape, labels.shape

In [ ]:
# using deep learning models
def plot_history(network_history):
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(network_history.history['loss'])
    plt.plot(network_history.history['val_loss'])
    plt.legend(['Training','Validation'])
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.plot(network_history.history['acc'])
    plt.plot(network_history.history['val_acc'])
    plt.legend(['Training','Validation'])
    plt.show()

In [ ]:
import tensorflow.keras.backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(512, kernel_size=5, strides=1,
                        padding="same", activation="relu",
                        input_shape=(X_train.shape[1], 1),
                        # kernel_regularizer=tf.keras.regularizers.L2(0.001)
                        ))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool1D(pool_size=5, strides=2, padding="same"))

model.add(tf.keras.layers.Conv1D(512, kernel_size=5, strides=1,
                        padding="same", activation="relu", 
                        # kernel_regularizer=tf.keras.regularizers.L2(0.001)
                                ))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool1D(pool_size=5, strides=2, padding="same"))

model.add(tf.keras.layers.Conv1D(256, kernel_size=5, strides=1,
                        padding="same", activation="relu",
                                 # kernel_regularizer=tf.keras.regularizers.L2(0.001)
                                ))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPool1D(pool_size=5, strides=2, padding="same"))

model.add(tf.keras.layers.Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu', 
                                 # kernel_regularizer=tf.keras.regularizers.L2(0.001)
                                ))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(tf.keras.layers.Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu', 
                                 # kernel_regularizer=tf.keras.regularizers.L2(0.001)
                                ))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides = 2, padding = 'same'))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu', 
                                # kernel_regularizer=tf.keras.regularizers.L2(0.001)
                               ))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(7, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc", f1_m])

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_acc",
    mode='auto',
    patience=5,
    restore_best_weights=True
)
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_acc',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001
)

EPOCHS = 100
batch_size = 64
history = model.fit(
    X_train,
    tf.keras.utils.to_categorical(y_train),
    validation_data=(X_cv, tf.keras.utils.to_categorical(y_cv)),
    epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[early_stopping_callback, reduce_lr_callback]
)
plot_history(history)
print("Accuracy of our model on test data : " , model.evaluate(X_test,tf.keras.utils.to_categorical(y_test))[1]*100 , "%")


In [ ]:
print("Accuracy of our model on test data : " , model.evaluate(X_test,tf.keras.utils.to_categorical(y_test))[1]*100 , "%")

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred


In [ ]:
y_check = np.argmax(tf.keras.utils.to_categorical(y_test), axis=1)
y_check

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_check, y_pred=y_pred)

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')